In [1]:
import io

import pandas as pd
from Bio import Entrez, SeqIO
from mysql.connector import connection
from sshtunnel import SSHTunnelForwarder

# Укажите ваш email (обязательно для использования Entrez)
Entrez.email = "your.email@example.com"

In [2]:
with open("db_curated_server_info.txt", "r") as file:
    lines = file.readlines()

config = {}

for line in lines:
    line = line.strip()
    if line and not line.startswith("#"):
        key, value = line.split("=", 1)
        config[key] = value.strip()

server_name = config.get("server_name")
srever_port = int(config.get("srever_port"))
ssh_password = config.get("ssh_password")
ssh_username = config.get("ssh_username")
db_adress = config.get("db_adress")
db_port = int(config.get("db_port"))

In [3]:
tunnel = SSHTunnelForwarder(
    (server_name, srever_port),
    ssh_password=ssh_password,
    ssh_username=ssh_username,
    remote_bind_address=(db_adress, db_port),
)
tunnel.start()
print(tunnel.local_bind_port)

46327


In [4]:
conn = connection.MySQLConnection(
    user="db_user",
    password="db_password",
    host="localhost",
    port=tunnel.local_bind_port,
    database="db_name",
)
cursor = conn.cursor()

In [5]:
query = "SHOW TABLES;"
cursor.execute(query)
cursor.fetchall()

[('alternative_name',),
 ('histone',),
 ('histone_description',),
 ('histone_has_publication',),
 ('publication',),
 ('sequence',),
 ('sequence_has_publication',)]

In [6]:
# add_histone = (
#     "INSERT INTO histone "
#     "(id, level, taxonomic_span, taxonomic_span_id, description, parent) "
#     "VALUES (%(id)s, %(level)s, %(taxonomic_span)s, %(taxonomic_span_id)s, %(description)s, %(parent)s)"
# )
# add_histone_description = (
#     "INSERT INTO histone_description "
#     "(summary, taxonomy, genes, evolution, expression, knock_out, function, sequence, localization, deposition, structure, interactions, disease, caveats) "
#     "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
# )
add_publication = (
    "INSERT INTO publication "
    "(id, title, doi, author, year) "
    "VALUES (%(id)s, %(title)s, %(doi)s, %(author)s, %(year)s)"
)
# add_sequence = (
#     "INSERT INTO sequence "
#     "(accession, variant, gi, ncbi_gene_id, hgnc_gene_name, taxonomy_id, organism, phylum, class, taxonomy_group, info, sequence, variant_under_consideration) "
#     "VALUES (%(accession)s, %(variant)s, %(gi)s, %(ncbi_gene_id)s, %(hgnc_gene_name)s, %(taxonomy_id)s, %(organism)s, %(phylum)s, %(class)s, %(taxonomy_group)s, %(info)s, %(sequence)s, %(variant_under_consideration)s)"
# )
# add_sequence_has_publication = (
#     "INSERT INTO sequence_has_publication "
#     "(sequence_accession, publication_id) "
#     "VALUES (%s, %s)"
# )
add_alternate_names = (
    "INSERT INTO alternative_name "
    "(name, taxonomy, gene, splice, histone) "
    "VALUES (%(name)s, %(taxonomy)s, %(gene)s, %(splice)s, %(histone)s)"
)
add_histone_has_publication = (
    "INSERT INTO histone_has_publication "
    "(histone_id, publication_id) "
    "VALUES (%s, %s)"
)

# To Do

## <span style="color:green">Add to structure description of cenH3_(Homo_sapiens):</span>

    ```
    CENP-A confers enhanced flexibility and elasticity to nucleosomes, influencing centromeric chromatin organization and ensuring accurate chromosome segregation during mitosis [fedulova_molecular_2024, pitman_minimal_2020, melters_intrinsic_2019]. Partial nucleosome unwrapping in CENP-A-containing nucleosomes is promoted by its shortened αN-helix [roulland_flexible_2016]. Cryo-EM analysis of human CENP-A nucleosomes revealed asymmetric DNA end flexibility, with the right DNA end being highly dynamic and partially detached from the histone core while the left end remained stably bound, a sequence-dependent behavior confirmed by molecular dynamics simulations [kono_free_2019, boopathi_phase-plate_2020]. Notably, this intrinsic asymmetry can be overcome by antibody stabilization. The single-chain antibody fragment PL2-6 symmetrically stabilizes both DNA ends through allosteric interactions, despite the inherent sequence-dependent asymmetry [zhou_atomic_2019, dogan_cenp-nucleosome_2021]. Molecular dynamics simulations revealed that CENP-A-containing tetramers exhibit enhanced stability but distinct dynamics compared to canonical octameric structures, explaining the unusual properties of CENP-A nucleosomes [zhao_oligomerization_2019].
    ```
    
> **Комментарии:**
> 1. **CENP-A придает нуклеосомам повышенную гибкость** – из-за укороченной αN-спирали концы ДНК легче раскручиваются, что важно для работы центромер `[roulland_flexible_2016]`. *Комментарий: исследование на клеточной линии **HeLa**, при этом в дискусси говорят о том, что скорее всего так и у других млекопитающих*  
> 2. **Асимметрия ДНК-динамики** – на последовательности 601 один конец ДНК в CENP-A нуклеосоме чаще раскручен, а другой остается стабильным `[kono_free_2019, boopathi_phase-plate_2020]`. *Комментарий: в первом исследовании использовалась **человеческая** модель, во втором - реконструированные in vitro нуклеосомы **человека** с CENP-A и ДНК Widom 601*  
> 3. **Антитела могут стабилизировать оба конца** – например, PL2-6 симметрично фиксирует ДНК, несмотря на исходную асимметрию, за счет аллостерических взаимодействий `[zhou_atomic_2019, dogan_cenp-nucleosome_2021]`. *Комментарий: первое исследование проводилось для **человеческого** CENP-A, второе - на **синтетической** модели (человек+лягушка)*  
> 4. **CENP-A нуклеосомы более эластичны**, чем канонические (H3), а гетеротипические (CENP-A/H3) занимают промежуточное положение `[melters_intrinsic_2019, pitman_minimal_2020]`. *Комментарий: первое исследование проводилось на клеточной линии **HeLa** с использованием **крысиного CENP-С фрагмента**, во втором - исследовались **человеческие** онкогенные нуклеосомы*  
> 5. **(CENP-A/H4)₂ тетрамер стабильнее (H3/H4)₂**, но его динамика отличается, что объясняет необычные свойства CENP-содержащих нуклеосом `[zhao_oligomerization_2019]`. *Комментарий: в исследовании использовалась **человеческая** модель*  


# <span style="color:green">Написать полное описание cenH3</span>

## Description

**summary:** cenH3 is a centromere-specific histone variant, which replaces canonical H3 in centromeric nucleosomes. It is required for kinetochore formation, mitotic progression and chromosome segregation.

**taxonomy:** cenH3 present in most eukaryotes. However, it has been lost in trypanosomes, the fungus Mucor, in four clades of holocentric insects and kinetoplastids [talbert_histone_2021, senaratne_formation_2021, sridhar_kinetochore_2022].

**genes:** 

**evolution:** The co-occurrence of CenH3 loss and holocentricity in insects suggests that the evolutionary development of holocentromeres in this group may have been facilitated by the loss of a CenH3-defined centromere [senaratne_formation_2021]. It has been demonstrated that CenH3 has undergone adaptive evolution at a significantly higher frequency in clades with asymmetric meiosis compared to those with symmetric meiosis [zedek_cenh3_2016]. This finding is in accordance with the predictions of the centromere drive model. Furthermore, the occurrence of more frequent adaptive changes in CenH3 is necessary to offset the negative effects of centromere drive [zedek_cenh3_2016].

**expression:** 

**knock_out:** 

**function:** Despite the high diversity of cenH3 proteins, they represent a key factor involved in centromere identity across a wide range of organisms. Moreover, it has been demonstrated that cenH3 from the monocot Zea mays is capable of substituting for cenH3 in A. thaliana [maheshwari_naturally_2015]. This example illustrates that the functions of the cenH3 histone are conserved across a broad evolutionary spectrum [maheshwari_naturally_2015].

**sequence:** cenH3 has an extended L1-loop and its N-terminal tail is very different from other H3 variants. cenH3s have an extended L1-loop and usually replace Phe84 in canonical H3 with Trp, and Thr 107 with Ala, Cys, or Ser. cenH3s ususally lack a conserved glutamine in the alpha1 helix of the histone fold. cenH3s typically have only about 50-60% amino acid identity to canonical H3 in the histone fold domain and no conservation of the N-terminus.

**localization:** 

**deposition:** Про шапероны

**structure:** 

**interactions:** Про кинетохору (есть статья с моделями сборки кинетохоры в разных видах живых организмов)

**disease:** CenH3 plays pro-viral and restriction role in tombusvirus replication in plants and in the yeast model host [gonzalez_centromeric_2022].

**caveats:** Разобраться с тем, как связаны cenH3 с транскрипцией центромерной ДНК [[Diverse mechanisms of centromere specification](https://www.notion.so/Diverse-mechanisms-of-centromere-specification-ff20544bd29e46138b5c696198d7857e?pvs=21) ].

1. Экспресия CENP-A происходит в раннюю G1 фазу. Вместе с тем, активно экспрессируется и шаперон HJURP, который встраивает CENP-A в нуклеосомы.
2. HJURP важен не просто для вставки CENP-A в нуклеосому, но и для определения локализации центромеры. Это происходит благодаря взаимодействию между HJURP и плотным комплексом, который состоит из белков Mis18α, Mis18β и M18BP1. Можно сказать, что комплекс Mis18 првлекает к себе HJURP, который в свою очередь уже связан с тетрамером CENP-A:H4.

# Add to structure description of cenH3_(Homo_sapiens):

    ```
    CENP-A confers enhanced flexibility and elasticity to nucleosomes, influencing centromeric chromatin organization and ensuring accurate chromosome segregation during mitosis [fedulova_molecular_2024, pitman_minimal_2020, melters_intrinsic_2019]. Partial nucleosome unwrapping in CENP-A-containing nucleosomes is promoted by its shortened αN-helix [roulland_flexible_2016]. Cryo-EM analysis of human CENP-A nucleosomes revealed asymmetric DNA end flexibility, with the right DNA end being highly dynamic and partially detached from the histone core while the left end remained stably bound, a sequence-dependent behavior confirmed by molecular dynamics simulations [kono_free_2019, boopathi_phase-plate_2020]. Notably, this intrinsic asymmetry can be overcome by antibody stabilization. The single-chain antibody fragment PL2-6 symmetrically stabilizes both DNA ends through allosteric interactions, despite the inherent sequence-dependent asymmetry [zhou_atomic_2019, dogan_cenp-nucleosome_2021]. Molecular dynamics simulations revealed that CENP-A-containing tetramers exhibit enhanced stability but distinct dynamics compared to canonical octameric structures, explaining the unusual properties of CENP-A nucleosomes [zhao_oligomerization_2019].
    ```

In [6]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='cenH3_(Homo_sapiens)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,cenH3_(Homo_sapiens),variant,Homo sapiens,9606,158,cenH3_(Mammalia),158,cenH3(Homo_sapiens) is a centromere-specific h...,null,null,...,CENP-A depletion at different stages of the ce...,CENP-A nucleosomes are essential for chromosom...,null,CENP-A in human chromosomes usually localized ...,Deposition of CENP-A into centromeric nucleoso...,null,CENP-A nucleosomes stabilize the inner kinetoc...,null,null,None


In [9]:
structure_desc = "CENP-A confers enhanced flexibility and elasticity to nucleosomes, influencing centromeric chromatin organization and ensuring accurate chromosome segregation during mitosis [fedulova_molecular_2024, pitman_minimal_2020, melters_intrinsic_2019]. Partial nucleosome unwrapping in CENP-A-containing nucleosomes is promoted by its shortened αN-helix [roulland_flexible_2016]. Cryo-EM analysis of human CENP-A nucleosomes revealed asymmetric DNA end flexibility, with the right DNA end being highly dynamic and partially detached from the histone core while the left end remained stably bound, a sequence-dependent behavior confirmed by molecular dynamics simulations [kono_free_2019, boopathi_phase-plate_2020]. Notably, this intrinsic asymmetry can be overcome by antibody stabilization. The single-chain antibody fragment PL2-6 symmetrically stabilizes both DNA ends through allosteric interactions, despite the inherent sequence-dependent asymmetry [zhou_atomic_2019, dogan_cenp-nucleosome_2021]. Molecular dynamics simulations revealed that CENP-A-containing tetramers exhibit enhanced stability but distinct dynamics compared to canonical octameric structures, explaining the unusual properties of CENP-A nucleosomes [zhao_oligomerization_2019]."
query = f"UPDATE histone_description SET structure='{structure_desc}' WHERE id=158"
print(query)
cursor.execute(query)

UPDATE histone_description SET structure='CENP-A confers enhanced flexibility and elasticity to nucleosomes, influencing centromeric chromatin organization and ensuring accurate chromosome segregation during mitosis [fedulova_molecular_2024, pitman_minimal_2020, melters_intrinsic_2019]. Partial nucleosome unwrapping in CENP-A-containing nucleosomes is promoted by its shortened αN-helix [roulland_flexible_2016]. Cryo-EM analysis of human CENP-A nucleosomes revealed asymmetric DNA end flexibility, with the right DNA end being highly dynamic and partially detached from the histone core while the left end remained stably bound, a sequence-dependent behavior confirmed by molecular dynamics simulations [kono_free_2019, boopathi_phase-plate_2020]. Notably, this intrinsic asymmetry can be overcome by antibody stabilization. The single-chain antibody fragment PL2-6 symmetrically stabilizes both DNA ends through allosteric interactions, despite the inherent sequence-dependent asymmetry [zhou_ato

In [10]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='cenH3_(Homo_sapiens)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,cenH3_(Homo_sapiens),variant,Homo sapiens,9606,158,cenH3_(Mammalia),158,cenH3(Homo_sapiens) is a centromere-specific h...,null,null,...,CENP-A depletion at different stages of the ce...,CENP-A nucleosomes are essential for chromosom...,null,CENP-A in human chromosomes usually localized ...,Deposition of CENP-A into centromeric nucleoso...,CENP-A confers enhanced flexibility and elasti...,CENP-A nucleosomes stabilize the inner kinetoc...,null,null,None


In [12]:
# Make sure data is committed to the database
conn.commit()

# Написать полное описание cenH3

**summary:** cenH3 is a centromere-specific histone variant, which replaces canonical H3 in centromeric nucleosomes. It is required for kinetochore formation, mitotic progression and chromosome segregation.

**taxonomy:** cenH3 present in most eukaryotes. However, it has been lost in trypanosomes, the fungus Mucor, in four clades of holocentric insects and kinetoplastids [talbert_histone_2021, senaratne_formation_2021, sridhar_kinetochore_2022]. 

**evolution:** The co-occurrence of CenH3 loss and holocentricity in insects suggests that the evolutionary development of holocentromeres in this group may have been facilitated by the loss of a CenH3-defined centromere [senaratne_formation_2021]. It has been demonstrated that CenH3 has undergone adaptive evolution at a significantly higher frequency in clades with asymmetric meiosis compared to those with symmetric meiosis [zedek_cenh3_2016]. This finding is in accordance with the predictions of the centromere drive model. Furthermore, the occurrence of more frequent adaptive changes in CenH3 is necessary to offset the negative effects of centromere drive [zedek_cenh3_2016].

**function:** Despite the high diversity of cenH3 proteins, they represent a key factor involved in centromere identity across a wide range of organisms. Moreover, it has been demonstrated that cenH3 from the monocot Zea mays is capable of substituting for cenH3 in A. thaliana [maheshwari_naturally_2015]. This example illustrates that the functions of the cenH3 histone are conserved across a broad evolutionary spectrum [maheshwari_naturally_2015].

**sequence:** cenH3 has an extended L1-loop and its N-terminal tail is very different from other H3 variants. cenH3s have an extended L1-loop and usually replace Phe84 in canonical H3 with Trp, and Thr 107 with Ala, Cys, or Ser. cenH3s ususally lack a conserved glutamine in the alpha1 helix of the histone fold. cenH3s typically have only about 50-60% amino acid identity to canonical H3 in the histone fold domain and no conservation of the N-terminus.

**disease:** CenH3 plays pro-viral and restriction role in tombusvirus replication in plants and in the yeast model host [gonzalez_centromeric_2022].


In [13]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='cenH3'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,cenH3,variant_group,null,null,96,H3,96,cenH3 is a centromere-specific histone variant...,null,null,...,null,null,null,null,null,null,null,null,null,None


In [15]:
summary_desc = "cenH3 is a centromere-specific histone variant, which replaces canonical H3 in centromeric nucleosomes. It is required for kinetochore formation, mitotic progression and chromosome segregation."
taxonomy_desc = "cenH3 present in most eukaryotes. However, it has been lost in trypanosomes, the fungus Mucor, in four clades of holocentric insects and kinetoplastids [talbert_histone_2021, senaratne_formation_2021, sridhar_kinetochore_2022]."
evolution_desc = "The co-occurrence of CenH3 loss and holocentricity in insects suggests that the evolutionary development of holocentromeres in this group may have been facilitated by the loss of a CenH3-defined centromere [senaratne_formation_2021]. It has been demonstrated that CenH3 has undergone adaptive evolution at a significantly higher frequency in clades with asymmetric meiosis compared to those with symmetric meiosis [zedek_cenh3_2016]. This finding is in accordance with the predictions of the centromere drive model. Furthermore, the occurrence of more frequent adaptive changes in CenH3 is necessary to offset the negative effects of centromere drive [zedek_cenh3_2016]."
function_desc = "Despite the high diversity of cenH3 proteins, they represent a key factor involved in centromere identity across a wide range of organisms. Moreover, it has been demonstrated that cenH3 from the monocot Zea mays is capable of substituting for cenH3 in A. thaliana [maheshwari_naturally_2015]. This example illustrates that the functions of the cenH3 histone are conserved across a broad evolutionary spectrum [maheshwari_naturally_2015]."
sequence_desc = "cenH3 has an extended L1-loop and its N-terminal tail is very different from other H3 variants. cenH3s have an extended L1-loop and usually replace Phe84 in canonical H3 with Trp, and Thr 107 with Ala, Cys, or Ser. cenH3s ususally lack a conserved glutamine in the alpha1 helix of the histone fold. cenH3s typically have only about 50-60% amino acid identity to canonical H3 in the histone fold domain and no conservation of the N-terminus."
disease_desc = "CenH3 plays pro-viral and restriction role in tombusvirus replication in plants and in the yeast model host [gonzalez_centromeric_2022]."
query = f"UPDATE histone_description SET summary='{summary_desc}', taxonomy='{taxonomy_desc}', evolution='{evolution_desc}', function='{function_desc}', sequence='{sequence_desc}', disease='{disease_desc}' WHERE id=96"
print(query)
cursor.execute(query)

UPDATE histone_description SET summary='cenH3 is a centromere-specific histone variant, which replaces canonical H3 in centromeric nucleosomes. It is required for kinetochore formation, mitotic progression and chromosome segregation.', taxonomy='cenH3 present in most eukaryotes. However, it has been lost in trypanosomes, the fungus Mucor, in four clades of holocentric insects and kinetoplastids [talbert_histone_2021, senaratne_formation_2021, sridhar_kinetochore_2022].', evolution='The co-occurrence of CenH3 loss and holocentricity in insects suggests that the evolutionary development of holocentromeres in this group may have been facilitated by the loss of a CenH3-defined centromere [senaratne_formation_2021]. It has been demonstrated that CenH3 has undergone adaptive evolution at a significantly higher frequency in clades with asymmetric meiosis compared to those with symmetric meiosis [zedek_cenh3_2016]. This finding is in accordance with the predictions of the centromere drive mode

In [16]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='cenH3'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,cenH3,variant_group,null,null,96,H3,96,cenH3 is a centromere-specific histone variant...,"cenH3 present in most eukaryotes. However, it ...",null,...,null,"Despite the high diversity of cenH3 proteins, ...",cenH3 has an extended L1-loop and its N-termin...,null,null,null,null,CenH3 plays pro-viral and restriction role in ...,null,None


In [17]:
# Make sure data is committed to the database
conn.commit()

# Close connections

In [18]:
cursor.close()
conn.close()
tunnel.stop()